In [ ]:
# Export a waveform to the Siglent SDG1032X arbitrary waveform generator using PyVISA.
# In this case a left 1kHz, right 2kHz FM stereo signal is created.
#
# Note that the SDG1032X has a bug, which regards any \x0a byte as a termination character and closes
# communication with the computer. This means that if a \x0a value is part of the waveform itself, 
# not the entire waveform is received by the SDG1032X. In many cases the SDG1032X will hang when an
# intermediate \x0a value is received. This bug appears when using SCPI. Designers of PyVISA seem
# to know this and coded around the bug, but in a way I do not understand.
#
# see: https://www.eevblog.com/forum/testgear/siglent-sdg1032x-bug-in-user-defined-waveforms/msg4677745/#msg4677745
#
#
# Using PyVISA does not 'crash' the SDG, but solves the issue in a worse manner.
# Also, I still do not understand this code completely.
# Why the four times larger waveform array than 16384?
# What about this sampling rate?
#
#
#
import time
import sys
import binascii
import matplotlib.pyplot as plt
import numpy as np
import pyvisa as visa

SAMPLES = 16384
SAMPLES = 0xfffe

# time axis
t = np.linspace(0,1,SAMPLES,endpoint=False)


f1 = 1
f2 = 2
f19 = 19
f38 = 38

s1 = np.sin(f1*2*np.pi*t)
s2 = np.sin(f2*2*np.pi*t)
s19 = np.sin(f19*2*np.pi*t)
s38 = np.sin(f38*2*np.pi*t)

# FM signal
ratio = .9
left = s1
right = s2
tone = s19
mono = (left + right) / 2
stereo = s38 * (left - right) / 2
signal = ratio * (mono + stereo) + (1 - ratio) * tone

try:
    rm = visa.ResourceManager()

    # Connect to device (Make sure to change the resource locator!)
    device = rm.open_resource('TCPIP::169.254.1.177::INSTR',query_delay=0.25)
except:
    print('Failed to connect to device...')
    sys.exit(0)
    
# prevent time-out errors by increasing the timeout to 10 seconds
device.timeout = 10000

# Create an empty array with 16384 points
WAVE = (signal * 32767).astype(int)

# Sample Rate in S/s
SAMPLE_RATE = 1638400

# Calculate factor for normalized frequency
F_FACTOR = (np.pi/(2*SAMPLE_RATE))

# Write Waveform to Device
# Note: byte order = little-endian!
device.write_binary_values('C1:WVDT WVNM,MUX_PYVISA2,FREQ,1000,AMPL,1.0,OFST,0.0,PHASE,0.0,WAVEDATA,', WAVE, datatype='i', is_big_endian=False)

device.write("C1:ARWV NAME,MUX_PYVISA2")

#Enable true Arb functionality
device.write("C1:SRATE MODE,TARB,VALUE,%f,INTER,LINE" % SAMPLE_RATE)

plt.plot(signal)